<h1 style=\"text-align: center; font-size: 50px;\"> <h1 style=\"text-align: center; font-size: 50px;\"> 📦 Register Model </h1> </h1>

📘 Project Overview: 
 This notebook demonstrates a modular architecture for answering natural language questions 
 over a GitHub Repo using only local and open-source models (e.g., LLaMA.cpp).
 The system processes long documents chunk-by-chunk and synthesizes a final answer using a multi-step LLM workflow.

# Notebook Overview

- Start Execution
- Define User Constants
- Install and Import Libraries
- Configure Settings
- Verify Assets
- KV Memory
- LLM Setup
- State Model
- Node Functions
- Graph Definition
- Graph Visualization
- Generated Answer
- Message History

# Start Execution

In [1]:
# ─────── Standard Library Imports ───────
import os  # OS-level utilities like path and environment operations
import sys  # Access to interpreter variables and runtime configuration
import time  # Time-related functions
from pathlib import Path  # Object-oriented file system paths

# Extend sys.path to include parent directory for local module resolution
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# ─────── Local Application Imports ───────
from src.utils import (  # Core utilities for logging, LLM interaction, and schema generation
    display_image,
    get_response_from_llm,
    json_schema_from_type,
    log_timing,
    logger,
)

In [2]:
start_time = time.time()  
logger.info("Notebook execution started.")

# Define User Constants

In [3]:
TOPIC: str = "Classification Algorithms"  
QUESTION: str = "Which machine learning classification algorithms are used in this project?"
REPO_URL: str = "https://github.com/HPInc/AI-Blueprints" 
FOLDER_PATH: Path = Path("data-science/classification-with-svm") 

# Install and Import Libraries

In [4]:
%%time

%pip install -r ../requirements.txt --quiet

Note: you may need to restart the kernel to use updated packages.
CPU times: user 11.8 ms, sys: 10.3 ms, total: 22.1 ms
Wall time: 1.06 s


In [5]:
from __future__ import annotations  # Enables postponed evaluation of type annotations (PEP 563)

# ─────── Standard Library Imports ───────
import base64                        # Encoding and decoding binary data
import functools                     # Higher-order functions and decorators
import io                            # Core tools for stream handling
import json                          # JSON serialization and deserialization
import logging                       # Logging configuration
import multiprocessing               # Multi-process parallelism
import os                            # OS-level operations
import shutil                        # File and directory operations
import sys                           # System-specific parameters and functions
import time                          # Time tracking utilities
import warnings                      # Warning control
import zipfile                       # Archive file handling
from collections import namedtuple   # Lightweight immutable object containers
from pathlib import Path             # Filesystem path manipulation
from typing import (
    Any, Dict, List, Literal, Optional, TypedDict  # Static type hints
)


# ─────── Third-Party Package Imports ───────
import requests                      # HTTP requests for external resources
import mlflow  # Model tracking and serving framework
from mlflow.tracking import MlflowClient  # Interface to interact with MLflow tracking server for experiments, runs, and artifacts
import yaml  # YAML file parsing
from IPython.display import HTML, display, Markdown  # Rich output formatting in Jupyter environments
from tqdm import tqdm  # Progress bar for loops

# ─────── LangChain Core & Community Imports ───────
from langchain.docstore.document import Document  # Document abstraction
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Intelligent text splitting
from langchain_community.document_loaders import (  # Document loaders for different file types
    CSVLoader,
    PyPDFLoader,
    TextLoader,
    UnstructuredExcelLoader,
    UnstructuredMarkdownLoader,
    UnstructuredWordDocumentLoader,
)
from langchain_community.llms import LlamaCpp  # Integration for running LlamaCpp locally

# ─────── LangGraph Imports ───────
from langgraph.graph import END, START, StateGraph  # Constructs and controls stateful agent graphs

# ─────── Local Application-Specific Imports ───────
from src.agentic_feedback_model import AgenticFeedbackModel  # Core agent logic for feedback analysis
from src.simple_kv_memory import SimpleKVMemory  # In-memory store for agent state

# Configure Settings

In [6]:
# Suppress Python warnings
warnings.filterwarnings("ignore")

In [7]:
REPO_NAME = Path(REPO_URL.split('/')[-1] + '-main')

In [8]:
INPUT_PATH: Path = Path("../data/input/downloaded_repo") / REPO_NAME /  FOLDER_PATH

MEMORY_PATH: Path = Path("../data/memory")   

CONFIG_PATH: Path = Path("../configs/config.yaml") 

MODEL_PATH = "/home/jovyan/datafabric/meta-llama3.1-8b-Q8/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf"
CONTEXT_WINDOW = 8192
MAX_TOKENS = CONTEXT_WINDOW // 8
CHUNK_SIZE = CONTEXT_WINDOW // 2
CHUNK_OVERLAP = CHUNK_SIZE // 8  

EXPERIMENT_NAME = "AIStudio-Agentic-Github-Repo-Analyzer-with-LangGraph-Experiment"
RUN_NAME = "AIStudio-Agentic-Github-Repo-Analyzer-with-LangGraph-Run"
MODEL_NAME = "AIStudio-Agentic-Github-Repo-Analyzer-with-LangGraph-Model"

In [9]:
logger.info('Notebook execution started.')

## Verify Assets

In [10]:
def log_asset_status(asset_path: str, asset_name: str) -> None:
    """
    Logs the status of a given asset based on its existence.

    Parameters:
        asset_path (str): File or directory path to check.
        asset_name (str): Name of the asset for logging context.
    """
    if Path(asset_path).exists():
        logger.info(f"{asset_name} is properly configured.")
    else:
        logger.info(f"{asset_name} is not properly configured. Please ensure the required asset is correctly configured in your AI Studio project according to the README file.")

In [11]:
log_asset_status(
    asset_path=INPUT_PATH,
    asset_name="Input Data",
)
log_asset_status(
    asset_path=MODEL_PATH,
    asset_name="LLM",
)

# KV Memory

In [12]:
memory: SimpleKVMemory = SimpleKVMemory(MEMORY_PATH)
memory.set('dummy key', 'dummy value')

# Download the GitHub Repo

In [13]:
def download_github_repo(repo_url: str, output_dir: str = "../data/input/downloaded_repo") -> Path:
    """
    Download and extract a public GitHub repository as a zip file.

    Args:
        repo_url (str): The GitHub URL (e.g., https://github.com/HPInc/AI-Blueprints)
        output_dir (str): Directory to extract the repo contents into.

    Returns:
        Path: Path to the extracted root folder.
    """
    # Normalize URL
    repo_url = repo_url.rstrip("/")
    if not repo_url.startswith("https://github.com/"):
        raise ValueError("URL must be a valid GitHub repository URL.")

    # Extract user/repo name
    parts = repo_url.replace("https://github.com/", "").split("/")
    if len(parts) != 2:
        raise ValueError("URL must be in format: https://github.com/owner/repo")

    owner, repo = parts
    zip_url = f"https://github.com/{owner}/{repo}/archive/refs/heads/main.zip"

    print(f"📦 Downloading repo zip from: {zip_url}")
    response = requests.get(zip_url)
    response.raise_for_status()

    # Extract zip contents
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall(output_dir)

    # The zip structure is usually: output_dir/repo-main/
    extracted_path = Path(output_dir) / f"{repo}-main"
    print(f"✅ Repository extracted to: {extracted_path.resolve()}")

    return extracted_path

In [14]:
%%time

download_github_repo(REPO_URL)

📦 Downloading repo zip from: https://github.com/HPInc/AI-Blueprints/archive/refs/heads/main.zip
✅ Repository extracted to: /home/jovyan/AI-Blueprints/generative-ai/agentic-github-repo-analyzer-with-langgraph/data/input/downloaded_repo/AI-Blueprints-main
CPU times: user 1.03 s, sys: 786 ms, total: 1.81 s
Wall time: 27.8 s


PosixPath('../data/input/downloaded_repo/AI-Blueprints-main')

# Load Documents

In [15]:
class SafeTextLoader(TextLoader):
    def load(self) -> list[Document]:
        encodings = ["utf-8", "utf-16", "latin-1", "cp1252"]
        for enc in encodings:
            try:
                with open(self.file_path, encoding=enc) as f:
                    text = f.read()
                return [Document(page_content=text)]
            except Exception:
                continue
        raise ValueError(f"Failed to decode file: {self.file_path}")

class NotebookLoader:
    def __init__(self, file_path: str):
        self.file_path = file_path

    def load(self) -> list[Document]:
        try:
            with open(self.file_path, encoding="utf-8") as f:
                notebook = json.load(f)
            cells = notebook.get("cells", [])
            source = "\n".join("".join(cell.get("source", [])) for cell in cells)
            return [Document(page_content=source)]
        except Exception as e:
            raise ValueError(f"Failed to load notebook: {self.file_path}: {e}")

In [16]:
logger.info("📂 Scanning directory for documents: %s", INPUT_PATH)

supported_extensions = {
    ".txt": SafeTextLoader,
    ".csv": lambda path: CSVLoader(path, encoding="utf-8", csv_args={"delimiter": ","}),
    ".xlsx": UnstructuredExcelLoader,
    ".docx": UnstructuredWordDocumentLoader,
    ".pdf": PyPDFLoader,
    ".md": UnstructuredMarkdownLoader,
    ".py": SafeTextLoader,
    ".json": SafeTextLoader,
    ".yml": SafeTextLoader,
    ".yaml": SafeTextLoader,
    ".ipynb": NotebookLoader,
}


all_docs = []

for file_path in Path(INPUT_PATH).rglob("*"):
    if any(part.startswith(".") and part not in {".", ".."} for part in file_path.parts):
        continue

    if file_path.is_dir():
        continue

    ext = file_path.suffix.lower()
    loader_class = supported_extensions.get(ext)

    if loader_class:
        try:
            loader = loader_class(str(file_path))
            docs = loader.load()
            all_docs.extend(docs)
            logger.info("✅ Loaded %d docs from %s", len(docs), file_path.name)
        except Exception as e:
            logger.warning("❌ Failed to load %s: %s", file_path.name, e)
    else:
        logger.info("⚠️ Unsupported file type: %s", file_path.name)

In [17]:
INPUT_TEXT = '\n\n'.join([doc.page_content for doc in all_docs])

# MLflow Registration

In [18]:
# 1. Set MLflow tracking URI and experiment
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI", "/phoenix/mlflow"))
mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)
print(f"Using MLflow tracking URI: {mlflow.get_tracking_uri()}")
print(f"Experiment: {EXPERIMENT_NAME}")

2025/09/11 20:58:32 INFO mlflow.tracking.fluent: Experiment with name 'AIStudio-Agentic-Github-Repo-Analyzer-with-LangGraph-Experiment' does not exist. Creating a new experiment.


Using MLflow tracking URI: /phoenix/mlflow
Experiment: AIStudio-Agentic-Github-Repo-Analyzer-with-LangGraph-Experiment


In [19]:
%%time

# These should point to the actual files you're using for model and memory
MODEL_ARTIFACTS = {
    "model_path": str(MODEL_PATH),
    "memory_path": str(MEMORY_PATH),
    "config_path": str(CONFIG_PATH),
    "demo": "../demo",
}
 
# === Start MLflow run, log, and register ===
with mlflow.start_run(run_name=RUN_NAME) as run:
    print(f"🚀 Started MLflow run: {run.info.run_id}")

    # Log and register the model using the classmethod
    AgenticFeedbackModel.log_model(
        model_name=MODEL_NAME,
        model_artifacts=MODEL_ARTIFACTS
    )

logger.info(f"✅ Model '{MODEL_NAME}' successfully logged and registered.")

2025/09/11 20:58:32 INFO mlflow.models.signature: Inferring model signature from type hints


🚀 Started MLflow run: 7af09b5a2dfa4358b3a4749c7b7bae17


Successfully registered model 'AIStudio-Agentic-Github-Repo-Analyzer-with-LangGraph-Model'.
Created version '1' of model 'AIStudio-Agentic-Github-Repo-Analyzer-with-LangGraph-Model'.


CPU times: user 892 ms, sys: 19.4 s, total: 20.3 s
Wall time: 3min 42s


In [20]:
# 3. Retrieve the latest version from the Model Registry
client = MlflowClient()
versions = client.get_latest_versions(MODEL_NAME, stages=["None"])

if not versions:
    raise RuntimeError(f"No registered versions found for model '{MODEL_NAME}'.")
    
latest_version = versions[0].version
model_info = mlflow.models.get_model_info(f"models:/{MODEL_NAME}/{latest_version}")

logger.info(f"Latest registered version of '{MODEL_NAME}': {latest_version}")
logger.info(f"Signature: {model_info.signature}")

In [21]:
%%time

# 4. Load the model from the Model Registry
loaded_model = mlflow.pyfunc.load_model(model_uri=f"models:/{MODEL_NAME}/{latest_version}")
logger.info(f"Successfully loaded model '{MODEL_NAME}' version {latest_version} for inference.")

CPU times: user 1.19 s, sys: 2.59 s, total: 3.78 s
Wall time: 55.7 s


In [22]:
# 5. Run a sample inference using the loaded model
input_payload = [{"topic": TOPIC, "question": QUESTION, "input_text": INPUT_TEXT, }]

print("\n=== Running Sample Inference ===")
results = loaded_model.predict(input_payload)
result = results[0]


=== Running Sample Inference ===


🔁 Processing each chunk: 100%|██████████| 135/135 [02:55<00:00,  1.30s/it, group=✅ Chunk 135 response length: 28 chars] 


🔁 Processing each grouped chunk answers: 100%|██████████| 1/1 [00:10<00:00, 10.56s/it, group=🧠 Synthesized partial answer (1/1)]



🔚 === Final Answer ===

# 🧠 Synthesized partial answer (1/1)

# Machine Learning Classification Algorithms
     The machine learning classification algorithms mentioned or utilized in the project documentation are:

## 1. Train/Test Split
     This is a technique used to evaluate the performance of a machine learning model. It involves splitting the dataset into two parts: a training set and a test set.

## 2. StandardScaler
     This is a feature scaling algorithm used to scale the features of a dataset to have a mean of 0 and a standard deviation of 1. This is useful for many machine learning algorithms that assume features are on the same scale.

## 3. Logistic Regression (LR)
     Logistic regression is a type of regression analysis that is used to model the relationship between a dependent variable and one or more independent variables.

## 4. Linear Discriminant Analysis (LDA)
     Linear discriminant analysis (LDA) is a type of linear regression analysis that is used to model t

# Generated Answer

In [23]:
display(Markdown(result.answer))

# 🧠 Synthesized partial answer (1/1)

# Machine Learning Classification Algorithms
     The machine learning classification algorithms mentioned or utilized in the project documentation are:

## 1. Train/Test Split
     This is a technique used to evaluate the performance of a machine learning model. It involves splitting the dataset into two parts: a training set and a test set.

## 2. StandardScaler
     This is a feature scaling algorithm used to scale the features of a dataset to have a mean of 0 and a standard deviation of 1. This is useful for many machine learning algorithms that assume features are on the same scale.

## 3. Logistic Regression (LR)
     Logistic regression is a type of regression analysis that is used to model the relationship between a dependent variable and one or more independent variables.

## 4. Linear Discriminant Analysis (LDA)
     Linear discriminant analysis (LDA) is a type of linear regression analysis that is used to model the relationship between a dependent variable and one or more independent variables.

## 5. K-Nearest Neighbors (KNN)
     K-nearest neighbors (KNN) is a type of supervised learning algorithm that is used to classify objects based on their features.

## 6. Classification and Regression Trees (CART)
     Classification and regression trees (CART) is a type of decision tree algorithm that is used to classify objects based on their features.

## 7. Gaussian Naive Bayes (NB)
     Gaussian naive Bayes (NB) is a type of supervised learning algorithm that is used to classify objects based on their features.

## 8. Support Vector Machines (SVM)
     Support vector machines (SVM) is a type of supervised learning algorithm that is used to classify objects based on their features.

# Message History

In [24]:
print(result.messages)

[
    {
        "role": "developer",
        "content": "User submitted a question."
    },
    {
        "role": "user",
        "content": "Which machine learning classification algorithms are used in this project?"
    },
    {
        "role": "developer",
        "content": "\ud83e\udde0 Relevance check result:"
    },
    {
        "role": "assistant",
        "content": "yes"
    },
    {
        "role": "developer",
        "content": "\ud83e\udded No cached answer found for question: 'Which machine learning classification algorithms are used in this project?'"
    },
    {
        "role": "developer",
        "content": "\u270f\ufe0f Rewritten user question:"
    },
    {
        "role": "assistant",
        "content": "What machine learning classification algorithms are mentioned or utilized in the project documentation?"
    },
    {
        "role": "developer",
        "content": "\ud83e\udde9 Chunked 1 documents into 135 chunks (size=4096, overlap=256)"
    },
    {
       

In [25]:
end_time: float = time.time()
elapsed_time: float = end_time - start_time
elapsed_minutes: int = int(elapsed_time // 60)
elapsed_seconds: float = elapsed_time % 60

logger.info(f"⏱️ Total execution time: {elapsed_minutes}m {elapsed_seconds:.2f}s")
logger.info("✅ Notebook execution completed successfully.")

Built with ❤️ using [**HP AI Studio**](https://hp.com/ai-studio).